In [219]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import os
os.chdir('/Users/shannontran/Downloads') #changing directory to where my files are

In [ ]:
# Connecting to Mongo db

In [220]:
client = MongoClient('localhost', 27017)

In [221]:
db = client.proj2_mongo

In [ ]:
# Importing datasets

In [223]:
# students.csv
students_df = pd.read_csv('students.csv')
students_df = students_df.iloc[1:] #removes first row with datatypes
students_df.head()

students_df = students_df.astype({'student_id':int, 'total_credits':int}) #changing data types from str to int


,student_id,first_name,last_name,dept_code,total_credits,student_year,living_status
1,1,John,Doe,ECON,100,Senior,Off-campus
2,2,Jon,Kashmeeri,BUS,113,Senior,Off-campus
3,3,Mary,Jane,NUTR,25,Freshman,On-campus
4,4,Mark,Maguire,CS,95,Senior,On-campus
5,5,Jasmine,Wang,PSYC,23,Freshman,On-campus


In [ ]:
# in clubs 
in_club_df = pd.read_csv('in_club.csv')[1:]
in_club_df = in_club_df.astype({'student_id':int, 'club_id':int})

In [224]:
# clubs
club_df = pd.read_csv('club.csv')[1:]
club_df = club_df.astype({'club_id':int, 'num_members':int})

In [ ]:
# Converting to dict
students_dict = students_df.to_dict('records')
in_club_dict = in_club_df.to_dict('records')
club_dict = clubs_df.to_dict('records')


In [249]:
# Nests club into in_club
for i in range(0, len(in_club_dict)):
    club_id = in_club_dict[i]['club_id']
    club_info = club_df[club_df['club_id'] == club_id][['club_name']].iloc[0].to_dict()
    in_club_dict[i]['club_id'] = club_info
    #print(club_info)

In [251]:
# Nests student into in_club
for i in range(0, len(in_club_dict)):
    student_id = in_club_dict[i]['student_id']
    student_info = students_df[students_df['student_id'] == student_id][['first_name', 'last_name']].iloc[0].to_dict()
    in_club_dict[i]['student_id'] = student_info


In [ ]:
in_club_dict[1]

In [ ]:
# Connecting to MongoDB

In [253]:
in_club_collection = db.in_club # create collection

In [254]:
in_club_collection.insert_many(in_club_dict) #inserts nested dictionary into in_club collection

In [264]:
# Who are club presidents?
query1 = in_club_collection.find({'role':'President'}

In [265]:
import pprint #pretty print
for e in query1:
    pprint.pprint(e)

{'_id': ObjectId('6297c04557c9c4be73ede0c9'),
 'club_id': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'_id': ObjectId('6297c04557c9c4be73ede0cf'),
 'club_id': {'club_name': 'The Math Wizards'},
 'role': 'President',
 'student_id': {'first_name': 'Jon', 'last_name': 'Kashmeeri'}}
{'_id': ObjectId('6297c04557c9c4be73ede0d6'),
 'club_id': {'club_name': 'Computational Data Sciences'},
 'role': 'President',
 'student_id': {'first_name': 'Kennedy', 'last_name': 'Vorkin'}}
